In [3]:
import pandas as pd

# Load the datasets
collisions = 'C:/Users/loma5202/OneDrive - Kruvelab/PhD/courses/introduction_to_data_science/project/UK_road_casualty_collision_1979_2022.csv'
vehicles = 'C:/Users/loma5202/OneDrive - Kruvelab/PhD/courses/introduction_to_data_science/project/UK_road_casualty_vehicle_1979_2022.csv'

# Read the CSV files
collisions_data = pd.read_csv(collisions)
vehicles_data = pd.read_csv(vehicles)

C:\Users\loma5202\AppData\Local\Temp\ipykernel_28756\1714511162.py:8: DtypeWarning: Columns (0,2,15,16,35) have mixed types. Specify dtype option on import or set low_memory=False.
  collisions_data = pd.read_csv(collisions)
C:\Users\loma5202\AppData\Local\Temp\ipykernel_28756\1714511162.py:9: DtypeWarning: Columns (0,2,24,27) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles_data = pd.read_csv(vehicles)


In [4]:
# Remove unnecessary columns
cols_remove = ['location_easting_osgr', 'location_northing_osgr', 'longitude', 'latitude', 'police_force',
               'local_authority_district', 'local_authority_ons_district', 'local_authority_highway',
               'first_road_number', 'second_road_number', 'pedestrian_crossing_human_control',
               'trunk_road_flag', 'lsoa_of_accident_location']

cols_save = ['accident_index', 'accident_year', 'accident_reference', 'vehicle_reference', 'vehicle_type',
             'sex_of_driver', 'age_of_driver', 'engine_capacity_cc', 'age_of_vehicle']

collisions_data = collisions_data.drop(cols_remove, axis = 1)
vehicles_data = vehicles_data[cols_save]


In [5]:
# Joining the datasets on 'accident_index'
road_acc_data = pd.merge(collisions_data, vehicles_data, on = 'accident_index', how = 'inner')

In [6]:
# Remove unnecessary columns from combined data frame
cols_remove = ['accident_index', 'accident_year_x', 'accident_reference_x', 'accident_reference_y', 
               'vehicle_reference']

road_acc_data = road_acc_data.drop(cols_remove, axis = 1)

In [7]:
# Re-classifying vehicle_type (all motorcycles in one group, cars + taxi in one group, etc.)
# Creating a mapping function for the vehicle_type groups
def map_vehicle_type(value):
    if value in [2, 3, 4, 5, 97, 104, 105, 106]:
        return 2
    elif value in [8, 9, 108, 109]:
        return 8
    elif value in [20, 21, 113]:
        return 20
    elif value in [90, 98, 99]:
        return 90
    elif value in [10, 110]:
        return 10
    else:
        return value

# Applying the mapping function to the vehicle_type column
road_acc_data['vehicle_type_grouped'] = road_acc_data['vehicle_type'].apply(map_vehicle_type)

# Checking the modification by displaying the first few rows
road_acc_data[['vehicle_type', 'vehicle_type_grouped']].head(10)


,vehicle_type,vehicle_type_grouped
0,109,8
1,104,2
2,109,8
3,109,8
4,109,8
5,109,8
6,109,8
7,109,8
8,109,8
9,109,8
